## Prompt: 
### The company wants to predict customer retention. 

The data provided (data.csv) consists of a of a cohort of customer_ids with a first order_id (order_sequence = 1) in January 2014 and any subsequent order details. We consider a customer retained (order_sequence = 2) if they made a subsequent order from January 2014 to December 2014. 
Develop insights to find best predictors for retention and suggest ways to operationalize insights

Description of data names and data

### Questions to answer with Data:

1.	Build a predictive model to determine retention of a user. Discuss model approach, validity, and KPIs
2.	Briefly discuss how the compnay can leverage the insights gained from the model?

My Answers: Data analysis below this cell helped me get to
My answer to Q1: Summary: More info at bottom of notebook notebook. I chose logistic regression for speed even though the model performance is quite low.
I choose a random forest due to possibility of improvements with a lot of parameter tuning later on.  Model performance is low, at with just under 60% accuracy (and no analysis done on whether retained or not retained customers are being more mis-classified). I would definitely need more time for parameter tuning and adjustment.

My answer to Q2: Assuming there are no more variables for a customer, the model (with more improvements and tuning) can be used to automatically gauge whether a customer will come back. It’s possible that we should target a consumer with more promotions through email or through changes in website advertisements (in terms of frequency and/or depth) in order to ensure that they do come back and buy more.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score

random_seed_value = 1

C:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Data Cleaning / Exploration
Before I decide on a model, Let's look at the data format, shape, and summary statistics

In [8]:
df = pd.read_csv( 'data.csv')

In [9]:
df.head(3)

,order_date,category_name,product_name,order_sequence,revenue,units,customer_id,order_id
0,2-Jan-14,Gifts,Archive DVDs,1,17.48,1,34462512,3.450000e+14
1,6-Jan-14,Large Formats,11x14,1,38.34,6,75309835,7.530000e+14
2,3-Jan-14,Photo Books,12x12 Memory Book,1,54.54,1,91660572,9.170000e+14


In [10]:
df.shape

(89636, 8)

In [11]:
df.describe()

,order_sequence,revenue,units,customer_id,order_id
count,89636.000000,89636.000000,89636.000000,8.963600e+04,8.963600e+04
mean,4.456123,19.523892,22.709637,4.951769e+07,4.528585e+14
std,6.331564,35.721153,62.664103,2.879722e+07,3.075390e+14
min,1.000000,-206.150000,0.000000,8.656000e+03,6.736700e+04
25%,1.000000,3.935735,1.000000,2.427273e+07,1.780000e+14
50%,2.000000,7.990000,1.000000,4.898846e+07,4.440000e+14
75%,5.000000,23.380000,13.000000,7.419158e+07,7.170000e+14
max,93.000000,2761.990000,2974.000000,9.999470e+07,1.000000e+15


In [12]:
df.corr()

,order_sequence,revenue,units,customer_id,order_id
order_sequence,1.000000,-0.073886,-0.027249,-0.016984,-0.053879
revenue,-0.073886,1.000000,0.164429,0.002186,0.034280
units,-0.027249,0.164429,1.000000,-0.004011,-0.048096
customer_id,-0.016984,0.002186,-0.004011,1.000000,0.851578
order_id,-0.053879,0.034280,-0.048096,0.851578,1.000000


Let's ensure we have no abnormal data issues- nas and data types

In [13]:
df.isna().sum() 

order_date        0
category_name     0
product_name      0
order_sequence    0
revenue           0
units             0
customer_id       0
order_id          0
dtype: int64

In [14]:
df.dtypes

order_date         object
category_name      object
product_name       object
order_sequence      int64
revenue           float64
units               int64
customer_id         int64
order_id          float64
dtype: object

Looking relatively clean so far! 

Remove orders in 2015, since not relevant to the question

In [15]:
df = df.loc[df['order_date'].str.contains("14"), ].reset_index(drop = True)

Let's make an indicator for a customer who first ordered in january.

In [16]:
df['jan14_order'] = df['order_date'].str.contains("Jan")

Let's generate an indicator for first orders in january 2014. 

In [17]:
df['firstorder_jan14'] = ( df['jan14_order'] &  ( df['order_sequence'] == 1) )
customers_first_order_jan = df.loc[ df['firstorder_jan14'] == True, 'customer_id' ]

In [18]:
len(customers_first_order_jan)

31840

In [19]:
len(set(customers_first_order_jan))

22036

Obviously, customers order multiple items per order

In [20]:
set(df.customer_id) - set(customers_first_order_jan) 

set()

In [21]:
# looks like every customer in the list first ordered in january

In [22]:
df.isnull().sum()

order_date          0
category_name       0
product_name        0
order_sequence      0
revenue             0
units               0
customer_id         0
order_id            0
jan14_order         0
firstorder_jan14    0
dtype: int64

Let's look at the orders for one customer who, which I found when scrolling through the dataset in excel, bought multiple items

In [23]:
df.loc[ df['customer_id'] == 91660572, ]

,order_date,category_name,product_name,order_sequence,revenue,units,customer_id,order_id,jan14_order,firstorder_jan14
2,3-Jan-14,Photo Books,12x12 Memory Book,1,54.540000,1,91660572,9.170000e+14,True,True
3,3-Jan-14,Photo Books,Memorabilia Pocket,1,1.990000,0,91660572,9.170000e+14,True,True
4,3-Jan-14,Photo Books,Premium Content,1,4.990000,0,91660572,9.170000e+14,True,True
5,3-Jan-14,Prints,4x6,1,0.691361,23,91660572,9.170000e+14,True,True
6,3-Jan-14,Prints,5x7,1,0.910607,1,91660572,9.170000e+14,True,True
7,3-Jan-14,Prints,8x10,1,14.708032,4,91660572,9.170000e+14,True,True


I wonder why units of the photo book, premium content are 0. It says [here](https://support.shutterfly.com/s/article/Photo-Books-Memorabilia-Pocket-1) that they are add on items for the memory book. I definitely appreciate that units is not 1 then.

Now let's create an indicator for those who are retained. First let's find the max order value for each customer. 

In [24]:
max_cust_order_sequence =  df.groupby( 'customer_id', as_index = True).max()['order_sequence']
max_cust_order_sequence = max_cust_order_sequence.reset_index( drop = False)
max_cust_order_sequence.columns = ['customer_id', 'max_order_sequence']

What's the average number of orders across all customers? Let's get the maximum order sequence of each customer to do this:

In [25]:
df.groupby('customer_id', as_index = True)['order_sequence'].max().describe()

count    22036.000000
mean         2.032991
std          2.361412
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         61.000000
Name: order_sequence, dtype: float64

Over 50% of consumers don't come back!

In [26]:
df = pd.merge( df, max_cust_order_sequence, how ='left', on = 'customer_id')

In [27]:
# if the max number of the order sequence of a customer is > 1, then they were retained
# OR were a customer already in the dataset previously
df['retained'] = (df['max_order_sequence'] > 1 )

In [28]:
df['retained'].value_counts()

True     44401
False    18911
Name: retained, dtype: int64

In [29]:
len(set(df.order_id))

3745

While we continue to explore the dataset to figure out the model, Let's look at the categorical variables

In [30]:
df.category_name.value_counts()

Prints           21532
Photo Books      16981
Cards             6574
Gifts             6048
Calendars         4280
Home Decor        2577
Card Upsell       1706
Large Formats     1670
Stationery        1509
Shipping           189
Services           107
Unassigned          68
Other               63
Yearbooks            6
OTHER                2
Name: category_name, dtype: int64

In [31]:
df.product_name.value_counts()

4x6                           13680
8x8 Story Book                 6003
5x7                            3968
Premium Content                3909
8x11 Classic Book              3722
Wall Calendars                 3079
8x10                           2664
5x7 Flat Card (Premium)        2373
Magnets                        1812
Mugs                           1389
Address Labels                 1349
Card Trims                     1335
10x10 Photo Book                973
Greeting Cards                  901
Large Calendars                 815
12x12 Memory Book               796
4x8s                            792
Memorabilia Pocket              779
Mousepads                       753
11x14                           727
16x20                           683
Wallet                          642
Desk Art                        616
5x7s                            573
iPhone Case                     523
6x8 Stationery Card             470
3x5 Stationery Card             415
11x14 Photo Book            

While product and category were intially items that I was just going to put into the model, there clearly are a lot of combinations of both! Maybe not a good idea to just throw this column and the category column into pd.dummies and interpret results out of a regression. 

Since interpretation of predictors is a priority, if we do end up doing a logit or decision tree (or another interpretative model) due to the small sample sizes of shipping, services, other, OTHER, yearbooks, I wouldn't trust those values. I would combine these groups into larger groups if I had time

## Features in Final Model:
I will use a logistic regression and random forest due to easy of interpretability of the results. 

Features I'll put into the algorithm are:
* total revenue for customer in first sale - maybe users who who spend a certain amount are more likely to come back (not
* num of units per category type for customer in first sale - maybe users who buy certain categories are more likely to come back

Now let's subset the data on the first order.

In [32]:
df_order1 = df.loc[df['order_sequence'] == 1,]

no minority class!

In [33]:
df_order1['total_revenue'] = df_order1.groupby('customer_id', sort=False)['revenue'].transform('sum')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
df_order1.shape

(31840, 13)

Let's generate columns that will be the amount of units a customer bought in their first January order in each category

In [35]:
sum_by_category = df_order1.groupby( ['customer_id', 'category_name'], as_index = True).sum()['units'].reset_index( drop = False)
sum_by_category = sum_by_category.pivot(index='customer_id', columns='category_name', values='units')
# fill na with 0, since that means that the customer didn't buy anything in their first order
sum_by_category[ sum_by_category.isna() ] = 0
sum_by_category = sum_by_category.reset_index( drop = False )
#sum_by_category = sum_by_category.drop(1, axis= 0) #prevent possible multi collinearity with total revenue

In [36]:
sum_by_category.sum().astype(int)

category_name
customer_id     -2147483648
Calendars              4310
Card Upsell               0
Cards                162443
Gifts                  3466
Home Decor             1438
Large Formats          1113
Other                     0
Photo Books            7779
Prints               593310
Services               3682
Shipping                  0
Stationery              886
Yearbooks                41
dtype: int32

some variables sum up to close to zero, making a matrix that isn't invertible. let's drop them

In [37]:
sum_by_category= sum_by_category.drop(['Other', 'Card Upsell', 'Shipping', 'Yearbooks'], axis = 1)

Let's now manipulate the original dataframe of first orders so it's at the customer level. Then merge it to sum by category dataframe

In [38]:
df_order2 = df_order1.loc[  ~df_order1.customer_id.duplicated(), ]
df_order2 = df_order2[['customer_id', 'total_revenue', 'retained']]

I don't expect a ton of multi collinearity between revenue and category due to variation in revenue likely due to sales

In [75]:
df_final = pd.merge( sum_by_category.reset_index(drop = True), df_order2, how = 'left', on='customer_id')

In [76]:
x = df_final.drop(['retained', 'customer_id'] , axis = 1).reset_index(drop = True)
y = df_final['retained'].reset_index( drop = True)

x = x.reset_index( drop = True)
x = sm.add_constant(x)

First, I'd like to run the tool on the original Logit function in order to make it easier to interpret coefficients 

In [58]:
logit_noreg= sm.Logit(y, x).fit()
logit1_res = logit_noreg.params

Optimization terminated successfully.
         Current function value: 0.672764
         Iterations 5


In [42]:
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

In [59]:
logit_noreg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               retained   No. Observations:                22036
Model:                          Logit   Df Residuals:                    22025
Method:                           MLE   Df Model:                           10
Date:                Fri, 28 Jun 2019   Pseudo R-squ.:               0.0004660
Time:                        16:50:58   Log-Likelihood:                -14825.
converged:                       True   LL-Null:                       -14832.
                                        LLR p-value:                    0.1812
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.4193      0.019    -22.552      0.000      -0.456      -0.383
Calendars         0.0053      0.010      0.550      0.582      -0.014       0.024
Cards             0.0002      0.001      0.218      0.827      -0.001       0.002
Gifts            -0.0027      0.018     -0.148      0.882      -0.038       0.033
Home Decor       -0.0021      0.040     -0.052      0.959      -0.081       0.077
Large Formats    -0.0388      0.037     -1.059      0.290      -0.111       0.033
Photo Books       0.0188      0.027      0.705      0.481      -0.033       0.071
Prints            0.0005      0.000      2.748      0.006       0.000       0.001
Services         -0.0054      0.004     -1.411      0.158      -0.013       0.002
Stationery       -0.0356      0.036     -1.002      0.317      -0.105       0.034
total_revenue    -0.0001      0.001     -0.238      0.812      -0.001       0.001
=================================================================================
"""

In [60]:
def logit_to_prob(logit_results):
    #Takes a coefficient from the logit model and returns the hazard estimate.
    return np.exp(logit_results)/(1 + np.exp(logit_results))

In [61]:
logit1_res.apply(logit_to_prob)

const            0.396692
Calendars        0.501318
Cards            0.500042
Gifts            0.499335
Home Decor       0.499481
Large Formats    0.490308
Photo Books      0.504692
Prints           0.500130
Services         0.498646
Stationery       0.491093
total_revenue    0.499970
dtype: float64

For the unregularized logistic fit on all the data, probabilities of customer being retained returning are pretty similar across all variables. We of course don't have interaction terms, and that may help enrich how to interpret these results. The pseduo r^2 is very low which doesn't bode well for prediction

Now let's build a an algorithm that's just focused on prediction

In [71]:
#normalize the independent variables
stsca = StandardScaler()
stsca.fit(x)
x_norm = stsca.transform(x)

Kinda small dataframe so let's say 20000/5 = 40000 - 5 folds should be okay to have a lot of training datasets. With more time I would tune regularlization.

In [82]:
kf = KFold( n_splits=5, shuffle=True, random_state=random_seed_value)
clf = LogisticRegression(random_state=0, solver='lbfgs',max_iter = 1000,  multi_class='multinomial')
scores_logit = cross_val_score(clf, x, y, cv=kf, n_jobs=1)

In [83]:
np.mean(scores_logit)

0.599337189185494

In [80]:
kf_rf = KFold( n_splits=5, shuffle=True, random_state=random_seed_value)
clf_rf =  RandomForestClassifier()
scores_rf = cross_val_score(clf_rf, x, y, cv=kf_rf, n_jobs=1)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change 

In [81]:
np.mean(scores_rf)

0.5544567255442895

With more time I would do the following:

1. Add more variables
    1. Due to large amount of variables in 'product_type' variable, and possible multi-collinearity with it and 'category_type', I'm not using it in the interpretation regression. I would definitely use a combination of text analysis, knowledge of the company's items, as well as knowledge of s's consumer base to see if I can regroup the category and product columns into more higher level groups/characteristics. Ultimately having higher level groups can help the regressions have more data on what types of items consumers are buying as well as produce interpretable coeffs that can be useful when deciding which categories/product types to focus on. Whether this is actually feasible would definitely depend on the investment the company has in this level of user research. These are some ideas for groups we can split items into for analysis.
        * size of product - maybe larger size items are correlated with a more 'unique' item - then would be a 1 time purchaser. This would also depend on the category we are in. Maybe someone is more likely to buy a lot of large size photos vs not as likely to buy a lot of large pillows
        * product function (a novelty item like a an apron vs photo books - there may be different levels of novelty items as well.)
    2. Look into using order date variable for results. Maybe weekend users are different are different from weekday users/certain promos happen at certain times?
    3. Convert the # units of product/category into a percentage. Didn't do this to time constraints but there is likely high multi collinearity with revenue now which will affect the logistic regression interpretability and standard errors - it's possible that the effect sizes we're seeing are 0 with too high standard errors.   
2. Check outliers for each feature. For example, I'm sure there are a few anomaly users who could 'bias' our results and reduce extrapolation of this model to an average user. If they would end up coming anyway, it may not be worth adjusting marketing strategy to them. This would require a lot of consumer research. There are also maybe some categories that don't have a lot of sales, so we should take the interpretation of the coefficients of those categories with a grain of salt due to low sample size.
3. Consider other alogrithms for prediction. Naive bayes is an obvious alternative to logistic as it can give probabilities (sacrificing obvious dependence between variables, and maybe increasing bias). Neural networks are other alternatives that come to mind but sacrifice interpretability of model results (and can also overfit if not careful)
4. Parameter tuning (particularly of regularization parameter in logit; depth of branches, min splits, minimum number of samples for random forest leaves using random search

It would also be interesting to
* It would be to cut the data and see if certain factors effect customer retention beyond order 2 and order 3. We would be able to use information about subsequent orders then. If we had prior data, we could operationalize the dataset beyond these customers